In [ ]:
import os, requests
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [ ]:
url = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx"
basename = os.path.basename(url)
headers = {'accept':'*/*', 'user-agent': 'Mozilla/5.0 (X11; Linux armv7l) AppleWebKit/537.36 (KHTML, like Gecko) Raspbian Chromium/78.0.3904.108 Chrome/78.0.3904.108 Safari/537.36'}
print("INFO: getting remote xls file")
resp = requests.get(url, headers=headers)
if resp.status_code!=200:
    msg = f"{url} error: {resp.status_code}"
    raise Exception(msg)
with open(basename, 'wb') as fd:
    fd.write(resp.content)


In [ ]:
csvout = basename.replace(".xlsx",".csv")
cmd  = f'ssconvert --export-type=Gnumeric_stf:stf_csv {basename} {csvout}'
print(cmd)
os.system(cmd)

In [ ]:
df = pd.read_csv(csvout, header=6, na_values=['…'])
df = df.rename(columns={'Unnamed: 0':'date'})
# Convert directly with string manipulation
df['date'] = pd.to_datetime(df['date'].str.replace('M', '-') + '-01') + pd.offsets.MonthEnd(0)
fileout = "cmo-data-monthly.csv"
df.to_csv(fileout,index=False)
print(f"output {len(df)} lines to {fileout}")

In [ ]:
import numpy as np
def goldprice():
    df = pd.read_csv("data/gldhist.csv").rename(columns={"price":"gold"})
    df["date"] = pd.to_datetime(df["date"])+dt.timedelta(days=1)
    return df.set_index("date")
def compute_performance_metrics(cumulative_wealth, risk_free_rate):   
    if cumulative_wealth[-1]==0:
        return {'mu': np.nan,'sigma': np.nan,'sharpe': np.nan,'maxdd': -1}
    logret = np.diff(np.log(cumulative_wealth))
    annual_return = np.mean(logret)*12
    annual_vol = np.std(logret) * np.sqrt(12)
    # Sharpe ratio (annualized)
    sharpe = (annual_return-risk_free_rate) / annual_vol
    # Maximum drawdown
    running_max = np.maximum.accumulate(cumulative_wealth)
    drawdowns = (cumulative_wealth - running_max) / running_max
    max_drawdown = np.min(drawdowns)  # Most negative value
    return {
        'mu': annual_return,
        'sigma': annual_vol,
        'sharpe': sharpe,
        'maxdd': max_drawdown
    }
def metricsstr(metrics):
    if metrics["maxdd"]==-1:
        return 'Ruin †'
    return f'y={metrics["mu"]*100:.1f}%, v={metrics["sigma"]*100:.0f}%, s={metrics["sharpe"]:.2f}, mdd={metrics["maxdd"]*100:.0f}%'

 
grokgold = goldprice()
dfgold = df[["date","GOLD"]].set_index("date")
    
# March 1968: The Two-Tier System
# March 17, 1968: London gold market closed after massive speculative attacks
# Central banks agreed: Continue to settle at $35/oz among themselves
# Private market allowed: To float freely, creating a "two-tier" system
plt.plot(dfgold,label=f"world bank {metricsstr(compute_performance_metrics(dfgold['GOLD'], 0))}")
plt.plot(grokgold,label=f"grok {metricsstr(compute_performance_metrics(grokgold['gold'], 0))}")
plt.legend()
plt.yscale('log')
plt.grid(which="both",axis="y")
plt.grid(axis="x")
plt.show()